ATS 2020

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from tqdm import tqdm_notebook

In [3]:
import requests

In [4]:
import time

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC

In [23]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [24]:
driver = webdriver.Chrome(r'C:\Users\venug\Downloads\chromedriver.exe',options = options)

In [7]:
driver.get('https://www.abstractsonline.com/pp8/#!/8998/sessions/*/1')

In [25]:
ResultsButton = driver.find_element_by_xpath('//*[@id="filter"]/div/div/button[3]')

In [26]:
ResultsButton.click()

In [31]:
ids = BeautifulSoup(driver.page_source)

In [48]:
t =ids.find_all('h1',class_ = 'name')

In [ ]:
//*[@id="paginator"]/div/ul/li[7]/span

In [71]:
pg = driver.find_element_by_css_selector('#paginator > div > ul > li.next > span') #2 for page 1
pg.click()

- Get sessions in a dataframe/dictionary with session id as key along with objectives, descriptions and links of presentations, with a new row for each link
- Iterate through these rows and obtain information on each of the presentations.

Extract all SessionLinks

In [8]:
SessInfo = {}
IterationDict = {}
i = 0
driver.get('https://www.abstractsonline.com/pp8/#!/8998/sessions/*/1')
timeout = 500
locator = '//*[@id="filter"]/div/div/button[3]'
WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
time.sleep(1)
ResultsButton = driver.find_element_by_xpath('//*[@id="filter"]/div/div/button[3]')
ResultsButton.click()
prevurl = ''
currenturl = driver.current_url
time.sleep(1)
while prevurl!=currenturl:
    prevurl = currenturl
    print(currenturl)
    time.sleep(1)
    Data = BeautifulSoup(driver.page_source)
    Results = Data.find('ul',id = 'results')
#     print(Results)
    Sessions = Results.find_all('div',class_ = 'span9')
    for j in Sessions:
        
#         SessInfo['Title'] = j.find('span',class_ = 'title color-primary').text
#         SessInfo['url'] = 'https://www.abstractsonline.com/pp8/#!/8998/session/'+ j.find('h1',class_ = 'name')['data-id']
#         print(SessInfo)
        IterationDict[j.find('h1',class_ = 'name')['data-id']] = {}
        IterationDict[j.find('h1',class_ = 'name')['data-id']]['Title'] = j.find('span',class_ = 'title color-primary').text
        IterationDict[j.find('h1',class_ = 'name')['data-id']]['url'] = 'https://www.abstractsonline.com/pp8/#!/8998/session/'+ j.find('h1',class_ = 'name')['data-id']
        i+=1
    time.sleep(1)
    pg = driver.find_element_by_css_selector('#params > div:nth-child(1) > div > ul > li.next > span') #2 for page 1
    pg.click()
    currenturl = driver.current_url
#     print(currenturl)

https://www.abstractsonline.com/pp8/#!/8998/sessions/*/1
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/2
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/3
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/4
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/5
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/6
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/7
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/8
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/9
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/10
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/11
https://www.abstractsonline.com/pp8/#!/8998/sessions/*/12


In [10]:
driver.close()

In [9]:
len(IterationDict.keys())

575

In [11]:
IterationDict


{'2430': {'Title': ' Thematic Poster Session. C56. DEVELOPMENTAL AND GENETIC CONTRIBUTIONS TO LUNG DISEASE',
  'url': 'https://www.abstractsonline.com/pp8/#!/8998/session/2430'},
 '2136': {'Title': " Mini Symposium. C13. USING 'OMICS TO ENDOTYPE COPD",
  'url': 'https://www.abstractsonline.com/pp8/#!/8998/session/2136'},
 '2373': {'Title': ' Mini Symposium. C16. NTM DISEASE - NEW DATA AND CHANGING PARADIGMS',
  'url': 'https://www.abstractsonline.com/pp8/#!/8998/session/2373'},
 '2138': {'Title': ' Mini Symposium. B94. ADULT AND PEDIATRIC ASTHMA GENETIC STUDIES',
  'url': 'https://www.abstractsonline.com/pp8/#!/8998/session/2138'},
 '2232': {'Title': ' Thematic Poster Session. B42. ILD EPIDEMIOLOGY I',
  'url': 'https://www.abstractsonline.com/pp8/#!/8998/session/2232'},
 '2371': {'Title': ' Mini Symposium. B98. NEW DEVELOPMENTS IN LUNG MICROBIOME',
  'url': 'https://www.abstractsonline.com/pp8/#!/8998/session/2371'},
 '2251': {'Title': ' Mini Symposium. C94. MECHANICAL VENTILATION: IN

Extract SessionDetails and urls for presentations

In [14]:
driver = webdriver.Chrome(r'C:\Users\venug\Downloads\chromedriver.exe')

In [15]:
SessionDetails = {}

for _id in tqdm_notebook(IterationDict.keys()):
    iteration = 1
#     driver = webdriver.Chrome(r'C:\Users\venug\Downloads\chromedriver.exe')
    url = IterationDict[_id]['url']
    driver.get(url)
#     time.sleep(1)
    timeout = 500
    locator = '//*[@id="body"]/div/div[2]/div[2]/table'
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
    Data = BeautifulSoup(driver.page_source)
    Table = Data.find('div',class_ = 'span7 presentation-table')
    trs = Table.find('table',class_ = 'table').find_all('tr')
    try:
        Objectives = Data.find('div',class_ = 'span3 offset1').text
    except:
        Objectives = ""
    try:
        Description = Data.find('blockquote',class_ = 'description').text
    except:
        Description = ""
    Location = Data.find('table',class_ = 'table table-bordered session-meta')
    Date = Location.find_all('td')[0].text
    Venue = Location.find_all('td')[1].text
    for tr in trs:
        t = _id + '-' + str(iteration)
        SessionDetails[t] = {}
        SessionDetails[t]['Session Title'] = IterationDict[_id]['Title']
        SessionDetails[t]['Description'] = Description
        SessionDetails[t]['Objectives'] = Objectives
        SessionDetails[t]['PresentationUrl'] = 'https://www.abstractsonline.com/pp8/'+tr.find_all('td')[1].find('a')['href']
        SessionDetails[t]['Date'] = Date
        #SessionDetails[t]['Location'] = Location
        iteration+=1
#     driver.close()

In [16]:
len(SessionDetails)

9848

In [25]:
PresentationDetails = {}


for i in tqdm_notebook(SessionDetails.keys()):
    PresentationDetails[i]= {}
    PresentationDetails[i]['Session Title'] = SessionDetails[i]['Session Title'] 
    PresentationDetails[i]['Description'] = SessionDetails[i]['Description'] 
    PresentationDetails[i]['Objectives'] = SessionDetails[i]['Objectives']
    PresentationDetails[i]['PresentationUrl'] = SessionDetails[i]['PresentationUrl'] 
    PresentationDetails[i]['Date'] = SessionDetails[i]['Date']
#     PresentationDetails[i]['Location']  = SessionDetails[i]['Location'] 
    url = SessionDetails[i]['PresentationUrl']

#     url = SessionDetails[i]['PresentationUrl']
    driver.get(url)
#         driver.refresh()
    timeout = 500
    locator = '//*[@id="body"]/div/div[2]/div/dl/dd[2]'
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
    time.sleep(2.5)
    Data = BeautifulSoup(driver.page_source,'lxml')
    PresentationDetails[i]['Presentation Title'] = Data.find('h2',class_ ='color-primary').text
    PresentationDetails[i]['Presentation Time'] = Data.find('span',class_ = 'session-date').text
    PresentationDetails[i]['Presentation Location'] = Data.find('span',class_ = 'session-location').text
    ParticipantAbstract = Data.select('#body > div > div:nth-child(3) > div > dl')
    headers = ParticipantAbstract[0].find_all('dt')
    content = ParticipantAbstract[0].find_all('dd')
    for j in range(len(headers)):
        PresentationDetails[i][headers[j].text] = content[j].text
        

In [300]:
url


'https://www.abstractsonline.com/pp8/#!/8998/presentation/10877'

In [301]:
driver.current_url == url

True

In [239]:
PresentationDetails

{'676-1': {'Session Title': ' Outside Organization Session. L13. PREVENTING HOSPITALIZATIONS AND ED VISITS IN CHILDREN WITH ASTHMA',
  'Description': '\nDescription\nAddressing asthma control is imperative for reducing the large, unequally-distributed burden of disease. Reducing exposure to environmental triggers, which can be exacerbated after a natural disaster, are an important component of asthma control. Evidence-based interventions, such as guidelines-based medical management, smoking cessation, asthma self-management education, home visits, and asthma-friendly policies, support asthma control. \nA new CDC initiative, CCARE (Controlling Childhood Asthma Reducing Emergencies), aims to prevent a half-million pediatric asthma-related emergency department visits and hospitalizations in 5 years. To achieve this goal, CCARE will promote evidence-based interventions across sectors and linkages between programs and clinics.\n',
  'Objectives': '\n\n\nObjective 1\nlearn the national surve

In [199]:
t = list(SessionDetails.keys())
t[1973]

'107-9'

In [291]:
df = pd.DataFrame.from_dict(PresentationDetails,orient = 'index')

In [292]:
df

,Session Title,Description,Objectives,PresentationUrl,Date,Presentation Title,Presentation Time,Presentation Location,Participant,Abstract
676-1,Outside Organization Session. L13. PREVENTING...,\nDescription\nAddressing asthma control is im...,\n\n\nObjective 1\nlearn the national surveill...,https://www.abstractsonline.com/pp8/#!/8998/pr...,"May 18, 2020, 12:15 PM - 1:15 PM",\n \n\n ...,"May 18, 2020, 12:15 PM - 1:15 PM","\n PENNSYLVANIA CONVENTION CENTER, Room 201 C ...","Josephine Malilay, PhD, MPH. Centers for Disea...",There is no abstract associated with this pres...
676-2,Outside Organization Session. L13. PREVENTING...,\nDescription\nAddressing asthma control is im...,\n\n\nObjective 1\nlearn the national surveill...,https://www.abstractsonline.com/pp8/#!/8998/pr...,"May 18, 2020, 12:15 PM - 1:15 PM",\n \n\n ...,"May 18, 2020, 12:15 PM - 12:30 PM","\n PENNSYLVANIA CONVENTION CENTER, Room 201 C ...",,There is no abstract associated with this pres...
676-3,Outside Organization Session. L13. PREVENTING...,\nDescription\nAddressing asthma control is im...,\n\n\nObjective 1\nlearn the national surveill...,https://www.abstractsonline.com/pp8/#!/8998/pr...,"May 18, 2020, 12:15 PM - 1:15 PM",\n \n\n ...,"May 18, 2020, 12:30 PM - 12:45 PM","\n PENNSYLVANIA CONVENTION CENTER, Room 201 C ...","Kisten Cowan, MPH. Centers for Disease Control...",There is no abstract associated with this pres...
676-4,Outside Organization Session. L13. PREVENTING...,\nDescription\nAddressing asthma control is im...,\n\n\nObjective 1\nlearn the national surveill...,https://www.abstractsonline.com/pp8/#!/8998/pr...,"May 18, 2020, 12:15 PM - 1:15 PM",\n \n\n ...,"May 18, 2020, 12:45 PM - 1:00 PM","\n PENNSYLVANIA CONVENTION CENTER, Room 201 C ...","Josephine Malilay, PhD, MPH. Centers for Disea...",There is no abstract associated with this pres...
676-5,Outside Organization Session. L13. PREVENTING...,\nDescription\nAddressing asthma control is im...,\n\n\nObjective 1\nlearn the national surveill...,https://www.abstractsonline.com/pp8/#!/8998/pr...,"May 18, 2020, 12:15 PM - 1:15 PM",\n \n\n ...,"May 18, 2020, 1:00 PM - 1:15 PM","\n PENNSYLVANIA CONVENTION CENTER, Room 201 C ...","Kanta Devi. Sircar, PhD, MPH. Centers for Dise...",There is no abstract associated with this pres...
...,...,...,...,...,...,...,...,...,...,...
2599-4,Scientific Symposium. D90. HORIZONS IN EXTRAC...,\nDescription\nThe presentations will review r...,\n\n\nObjective 1\ninterpret recent clinical t...,https://www.abstractsonline.com/pp8/#!/8998/pr...,"May 20, 2020, 1:00 PM - 3:00 PM",\n \n\n ...,"May 20, 2020, 1:20 PM - 1:40 PM","\n PENNSYLVANIA CONVENTION CENTER, Terrace Bal...","Marie E. Steiner, MD, MS. University of Minnes...",There is no abstract associated with this pres...
2599-5,Scientific Symposium. D90. HORIZONS IN EXTRAC...,\nDescription\nThe presentations will review r...,\n\n\nObjective 1\ninterpret recent clinical t...,https://www.abstractsonline.com/pp8/#!/8998/pr...,"May 20, 2020, 1:00 PM - 3:00 PM",\n \n\n ...,"May 20, 2020, 1:40 PM - 2:00 PM","\n PENNSYLVANIA CONVENTION CENTER, Terrace Bal...","Cara Agerstrand, MD. Columbia University Medic...",There is no abstract associated with this pres...
2599-6,Scientific Symposium. D90. HORIZONS IN EXTRAC...,\nDescription\nThe presentations will review r...,\n\n\nObjective 1\ninterpret recent clinical t...,https://www.abstractsonline.com/pp8/#!/8998/pr...,"May 20, 2020, 1:00 PM - 3:00 PM",\n \n\n ...,"May 20, 2020, 2:00 PM - 2:20 PM","\n PENNSYLVANIA CONVENTION CENTER, Terrace Bal...","Jenelle Holst. Badulak, MD. University of Wash...",There is no abstract associated with this pres...
2599-7,Scientific Symposium. D90. HORIZONS IN EXTRAC...,\nDescription\nThe presentations will review r...,\n\n\nObjective 1\ninterpret recent clinical t...,https://www.abstractsonline.com/pp8/#!/8998/pr...,"May 20, 2020, 1:00 PM - 3:00 PM",\n \n\n ...,"May 20, 2020, 2:20 PM - 2:40 PM","\n PENNSYLVANIA CONVENTION CENTER, Terrace Bal...","Ira

In [293]:
df['Description'] = df['Description'].str.replace('\n','')

In [294]:
df['Objectives'] = df['Objectives'].str.replace('\n','')
df['Presentation Title'] = df['Presentation Title'].str.replace('\n','').str.strip()
df['Presentation Location'] = df['Presentation Location'].str.replace('\n','')

In [295]:
df['Presentation Start Time'] = df['Presentation Time'].str.split(',',expand = True)[2].str.split(' - ',expand = True)[0]
df['Presentation End Time'] =  df['Presentation Time'].str.split(',',expand = True)[2].str.split(' - ',expand = True)[1]

In [296]:
df['Presentation Date'] = df['Presentation Time'].str.split(',',expand = True)[0]+', '+df['Presentation Time'].str.split(',',expand = True)[1]

In [297]:
df.to_excel('ATS2020Version2Offline.xls')